In [ ]:
%matplotlib inline

import numpy as np
import cv2
import matplotlib.pyplot as plt


In [ ]:
def fourier(file, filterfunc):
    img = cv2.imread(file, 0)

    ft = cv2.dft(np.float32(img), flags = cv2.DFT_COMPLEX_OUTPUT)
    ft_shift = np.fft.fftshift(ft)

    ut_shift = filterfunc(ft_shift.copy())

    ut_ishift = np.fft.ifftshift(ut_shift)
    ut = cv2.idft(ut_ishift)

    return cv2.magnitude(ut[:, :, 0], ut[:, :, 1])

In [ ]:
def highpath_filter(h_size):
    def filter(ft):
        w = ft.shape[1]
        h = ft.shape[0]
    
        cx = int(w / 2)
        cy = int(h / 2)

        return cv2.circle(ft.copy(), (cx, cy), h_size, [0, 0], -1)
    
    return filter


def lowpath_filter(l_size):
    def filter(ft):
        w = ft.shape[1]
        h = ft.shape[0]
    
        cx = int(w / 2)
        cy = int(h / 2)
    
        mask = np.zeros(ft.shape, np.uint8)
        mask = cv2.circle(mask, (cx, cy), l_size, [1, 1], -1)
        
        return ft * mask
    
    return filter


def bandpath_filter(h_size, l_size):
    high = highpath_filter(h_size)
    low = lowpath_filter(l_size)
    
    return lambda ft: high(low(ft))
    

In [ ]:
plt.imshow(fourier('img/01.jpg', highpath_filter(4)), cmap = 'gray')

In [ ]:
plt.imshow(fourier('img/01.jpg', lowpath_filter(20)), cmap = 'gray')

In [ ]:
plt.imshow(fourier('img/01.jpg', bandpath_filter(4, 20)), cmap = 'gray')